In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [74.2 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [923 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,019 kB]
Hit:13 http://ppa.launchpad.net/graphics

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-22 02:15:25--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.61MB/s    in 0.2s    

2023-03-22 02:15:26 (5.61 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [4]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
video_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
video_df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   49033728|R1P1G5KZ05H6RD|6302503213|     748506413|The Night They Sa...|           Video|          5|            0|          0|   N|                Y|    Very satisfied!!|Fast shipping. Pl...| 2015-08-31|
|         US|   17857748|R106N066IUN8ZV|B000059PET|     478710180|Hamlet / Kline, N...|           Video|          5|    

In [ ]:
# Get the number of rows in the DataFrame.
video_df.count()


1785997

# Transform the Data

## Create the "review_id_table".

In [5]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = video_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1P1G5KZ05H6RD|   49033728|6302503213|     748506413| 2015-08-31|
|R106N066IUN8ZV|   17857748|B000059PET|     478710180| 2015-08-31|
| R7WTAA1S5O7D9|   25551507|0788812807|     981002815| 2015-08-31|
|R32HFMVWLYOYJK|   21025041|6302509939|     333219811| 2015-08-31|
| RWT3H6HBVAL6G|   40943563|B00JENS2BI|     538101194| 2015-08-31|
|R1S3T3GWUGQTW7|   17013969|6305761302|     716303278| 2015-08-31|
|R3R0QYHA800REE|   47611685|6300157555|     134996462| 2015-08-31|
|R1FR0EQPHPW5NM|   35680737|6300189570|     498116870| 2015-08-31|
| RGORN81H45NI7|   10747909|B000SXQ5US|      77519275| 2015-08-31|
|R1CNYN4ABNOJSU|     126341|B00008F22G|     917778300| 2015-08-31|
|R2DW06821PMN40|   40676812|6303453961|     187850980| 2015-08-31|
|R1CS8AMA8B0VBJ|   19863533|6302453178|     901056605| 2015-08

## Create the "products" Table

In [7]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = video_df.select(["product_id", "product_title"]).dropDuplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0788806270|Bambi (Walt Disne...|
|078881107X|Kiki's Delivery S...|
|6303559018|To Catch a Yeti [...|
|6302844061|      Silkwood [VHS]|
|B00003XAMY|Wagons Roll at Ni...|
|0783216084|          Jaws [VHS]|
|6301966376|The Compleat Beat...|
|6303315429| Yoga for Life [VHS]|
|6302969204|Miracle in Milan ...|
|6301540441|As Summers Die [VHS]|
|6304611366|Loretta Young Sho...|
|1559838450|Hans Christian An...|
|6304263198|   Latcho Drom [VHS]|
|6301017250|Dead Don't Die [VHS]|
|6301782135|John Hammond: Fro...|
|6304925158|The Education of ...|
|6305403309| Dead Husbands [VHS]|
|6300152936|       Faeries [VHS]|
|B0000897E9|  Femme Fatale [VHS]|
|B0002X4MIQ|Touched By An Ang...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [8]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = video_df.groupby("customer_id").agg({"customer_id": "count"})
customers_df.show()

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   14862100|                 1|
|   45193257|               122|
|   14777050|                17|
|   34336061|                 1|
|   16459933|                 1|
|   26101727|                 1|
|    7174045|                 3|
|   33458400|                 1|
|   47687561|                 1|
|   46761666|                 1|
|   12314433|                 1|
|   32872653|                17|
|   41335277|                 1|
|   13959488|                 1|
|   13912262|                 1|
|   18892421|                 3|
|   27942166|                 1|
|   42458575|                 1|
|     307244|                 1|
|   13139879|                 4|
+-----------+------------------+
only showing top 20 rows



## Create the "vine_table".

In [9]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = video_df.select(["review_id", "star_rating", "helpful_votes","total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1P1G5KZ05H6RD|          5|            0|          0|   N|
|R106N066IUN8ZV|          5|            0|          0|   N|
| R7WTAA1S5O7D9|          4|            0|          0|   N|
|R32HFMVWLYOYJK|          5|            0|          0|   N|
| RWT3H6HBVAL6G|          3|            0|          0|   N|
|R1S3T3GWUGQTW7|          5|            0|          0|   N|
|R3R0QYHA800REE|          4|            0|          0|   N|
|R1FR0EQPHPW5NM|          4|            1|          2|   N|
| RGORN81H45NI7|          5|            1|          1|   N|
|R1CNYN4ABNOJSU|          5|            0|          0|   N|
|R2DW06821PMN40|          3|            0|          0|   N|
|R1CS8AMA8B0VBJ|          5|            0|          0|   N|
|R343CPRI4MC9J0|          5|            0|          0|   N|
| R3XP0G8P2BOTP|          5|            

# Load

In [10]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
config = {"user":"postgres", "password": "<password>", "driver":"org.postgresql.Driver"}

In [11]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table="review_id_df", mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table="products_df", mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table="customers_df", mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table="vine_df", mode=mode, properties=config)